# Echarts Pane Example
            

## Description

[ECharts](https://www.echartsjs.com/en/index.html) is an open-sourced JavaScript
visualization tool, which can run fluently on PC and mobile devices.
It is compatible with most modern Web Browsers. Its also an **Apache incubator project**. The library is very fast with a modern look and feel.

[Pyecharts](https://pyecharts.org/#/en-us/) is a Python api for using ECharts in Python
including Standalone, Flask, Django and Jupyter Notebooks.

Below we develop an `ECharts` pane capable of showing Echarts dicts and Pyecharts objects **enabling us to develop awesome analytics apps using the power of Echarts, Panel and Python**.

<a href="https://www.echartsjs.com/en/index.html" target="blank_"><img src="https://img2018.cnblogs.com/blog/1244003/201903/1244003-20190301152709535-1909606063.png" style="max-height:400px"></img></a>

**Author:**
[Marc Skov Madsen](https://datamodelsanalytics.com) ([awesome-panel.org](https://awesome-panel.org))

**Tags:**
[Panel](https://panel.holoviz.org/)
[Echarts](https://www.echartsjs.com/en/index.html)
[PyeChart](https://pyecharts.org/#/en-us/)
[Pane](https://panel.holoviz.org/user_guide/Components.html)
[WebComponent](https://panel.holoviz.org/reference/panes/WebComponent.html)
[Python](https://www.python.org/)

**License:**
[MIT](https://opensource.org/licenses/mit-license.php)

**Resources:**
[Echarts intro Video](https://www.youtube.com/watch?v=MF34Cgk5Rp0)

## Code

### Imports

In [ ]:
import param
import panel as pn

# Configure and import js in Notebook
ECHART_JS_NOTEBOOK = "https://pyecharts.github.io/jupyter-echarts/echarts/echarts.min.js"
pn.config.js_files["echart"]=ECHART_JS_NOTEBOOK
pn.extension()

# Configure js for server
ECHART_JS_SERVER = "https://cdn.bootcss.com/echarts/3.7.2/echarts.min.js"
pn.config.js_files["echart"]=ECHART_JS_SERVER

### HTML Example

In [ ]:
%%HTML

<div id="echarts-example" style="width: 100%;height:400px;"></div>
<script type="text/javascript">
    // based on prepared DOM, initialize echarts instance
    var myChart = echarts.init(document.getElementById('echarts-example'));

    // specify chart configuration item and data
    var option = {
        title: {
            text: 'ECharts Entry Example'
        },
        tooltip: {},
        legend: {
            data:['Sales']
        },
        xAxis: {
            data: ["shirt","cardign","chiffon shirt","pants","heels","socks"]
        },
        yAxis: {},
        series: [{
            name: 'Sales',
            type: 'bar',
            data: [5, 20, 36, 10, 10, 20]
        }]
    };

    // use configuration item and data specified to show chart
    myChart.setOption(option);
</script>

### `ECharts` Pane

In [ ]:
ECHARTS_HTML = """
    <div class="echart" style="width:100%;height:100%;"></div>
    <script type="text/javascript">
        var myScript = document.currentScript;
        var myDiv = myScript.parentElement.firstElementChild;
        var myChart = echarts.init(myDiv);
        myDiv.eChart = myChart;
        Object.defineProperty(myDiv, 'option', {
           get: function() { return null; },
           set: function(val) { this.eChart.setOption(val); this.eChart.resize();}
        });
        myDiv.after_layout = myChart.resize; // Resizes the chart after layout of parent element
    </script>"""

In [ ]:
class ECharts(pn.pane.WebComponent):
    html = param.String(ECHARTS_HTML)
    properties_to_watch = param.Dict({"option": "option"})

    echart = param.Parameter()
    option = param.Dict()

    def __init__(self, **params):
        if "echart" in params:
            params["option"] = self._to_echart_dict(params["echart"])
        super().__init__(**params)

    @classmethod
    def _to_echart_dict(cls, echart):
        if isinstance(echart, dict):
            return echart
        if 'pyecharts' in sys.modules:
            import pyecharts
            if isinstance(echart, pyecharts.charts.chart.Chart):
                return json.loads(echart.dump_options())

        return {}

    @param.depends("echart", watch=True)
    def update(self):
        self.option = self._to_echart_dict(self.echart)

## Output

### Simple App

In [ ]:
ECHART_DICT = {
    "title": {"text": "ECharts Entry example"},
    "tooltip": {},
    "legend": {"data": ["Sales"]},
    "xAxis": {"data": ["shirt", "cardign", "chiffon shirt", "pants", "heels", "socks"]},
    "yAxis": {},
    "series": [{"name": "Sales", "type": "bar", "data": [5, 20, 36, 10, 10, 20]}],
}
simple_app = pn.Column(
    ECharts(echart=ECHART_DICT, height=500),
    sizing_mode="stretch_width"
)
simple_app

### ECharts  App

In [ ]:
BOUNDS=(0,100)

class EchartsApp(param.Parameterized):
    title = param.String("Awesome Panel")
    plot_type = param.ObjectSelector("bar", objects=["bar", "scatter"], label="Plot Type")
    
    shirt = param.Integer(default=5, bounds=BOUNDS) 
    cardign = param.Integer(default=20, bounds=BOUNDS)
    chiffon_shirt = param.Integer(default=36, bounds=BOUNDS)
    pants = param.Integer(default=10, bounds=BOUNDS)
    heels = param.Integer(default=10, bounds=BOUNDS)
    socks = param.Integer(default=20, bounds=BOUNDS)
    
    def __init__(self, **params):
        super().__init__(**params)
        
        self.plot = ECharts(height=500)
        
        self.update_plot()
        
    @param.depends("title", "plot_type", "shirt", "cardign", "chiffon_shirt", "pants", "heels", "socks", watch=True)
    def update_plot(self):
        echart = {
                "title": {"text": self.title},
                "tooltip": {},
                "legend": {"data": ["Sales"]},
                "xAxis": {"data": ["shirt", "cardign", "chiffon shirt", "pants", "heels", "socks"]},
                "yAxis": {},
                "series": [
                    {
                        "name": "Sales", 
                        "type": self.plot_type, 
                        "data": [self.shirt, self.cardign, self.chiffon_shirt, self.pants, self.heels, self.socks],
                    }
                ],
        }
        self.plot.echart = echart
        
    def view(self):
        top_app_bar = pn.Row(
            pn.pane.PNG("https://www.echartsjs.com/en/images/logo.png", sizing_mode="fixed", height=40, margin=(15,0,5,25)),
            pn.layout.VSpacer(),
            pn.pane.PNG("https://panel.holoviz.org/_static/logo_horizontal.png", sizing_mode="fixed", height=40, margin=(15,0,5,25)),
            "",
            pn.layout.VSpacer(),
            background="rgb(41, 60, 85)",
            height=70
            )
        
        settings_pane  =pn.Param(
            self, 
            show_name=False, 
            width=200, 
            sizing_mode="stretch_height",
            background="rgb(245, 247, 250)"
        )
        
        return pn.Column(
            top_app_bar,
            pn.layout.HSpacer(height=50),
            pn.Row(
                self.plot,
                settings_pane, 
            ),
        )

pn.config.sizing_mode="stretch_width"
app =  EchartsApp()
view=app.view()
view

In [ ]:
def show_in_server(event):
    view.show()
    
show_button = pn.widgets.Button(name="Deploy to Server", button_type="success")
show_button.on_click(show_in_server)
show_button